In [1]:
import sys 
sys.path.append('/home/monte.flora/python_packages/wofs_ml_severe')
sys.path.append('/home/monte.flora/python_packages/WoF_post')
#sys.path.append('/home/samuel.varga/python_packages/wofs_ml_severe')
sys.path.append('/home/samuel.varga/projects/2to6_hr_severe_wx/experiments')
sys.path.append('/home/samuel.varga/python_packages/MontePython/')
import numpy as np
import xarray as xr
from glob import glob
from ml_2to6_data_pipeline import (GridPointExtracter,
                                                       subsampler, 
                                                       load_dataset)
from os.path import join
# Import packages 
import pandas as pd
import numpy as np

# Plotting code imports 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

# We add the github package to our system path so we can import python scripts for that repo. 
import sys
sys.path.append('/home/samuel.varga/projects/2to6_hr_severe_wx/')
from main.io import load_ml_data, load_bl_data
from bayeshist import bayesian_histogram, plot_bayesian_histogram
from wofs_ml_severe.data_pipeline.storm_report_loader import StormReportLoader
from wofs.plotting.util import decompose_file_path
from wofs.verification.lsrs.get_storm_reports import StormReports

In [2]:
FRAMEWORK='POTVIN'; TIMESCALE='0to3'
base_path = f'/work/samuel.varga/data/{TIMESCALE}_hr_severe_wx/{FRAMEWORK}'
X,y,metadata = load_ml_data(base_path=base_path, 
                            mode='test', 
                            target_col='hail_severe__36km',
                           FRAMEWORK=FRAMEWORK,
                           TIMESCALE=TIMESCALE) #What scales are available for targets: 9, 15, 36

In [3]:
dates=np.sort(pd.unique(metadata['Run Date'])) #Grab the dates in the testing set
init_time_='0000'
dates

array(['20170501', '20170508', '20170517', '20170518', '20170519',
       '20170524', '20170525', '20170530', '20180502', '20180504',
       '20180509', '20180512', '20180519', '20180524', '20180530',
       '20190513', '20190518', '20190521', '20190522', '20200520',
       '20200521', '20200526', '20200528', '20210506', '20210510',
       '20210513', '20210514', '20210521', '20210523', '20210524',
       '20210525'], dtype=object)

In [5]:
print('{:^8}|{}|{}|{}|'.format('Date','Wind','Hail','Torn'))
print('--------|----|----|----|')
for date in dates:
    indir = glob(f'/work/mflora/SummaryFiles/{date}/{init_time_}/wofs_ENS_24*')[0]
    # Get the storm reports. 
    comps = decompose_file_path(indir)
    init_time = comps['VALID_DATE']+comps['VALID_TIME']

    init_time = comps['VALID_DATE']+comps['INIT_TIME']



    report = StormReportLoader(initial_time=init_time, 
            forecast_length=180,
            err_window=15, 
            reports_path='/work/samuel.varga/data/2to6_hr_severe_wx/reports/STORM_EVENTS_{0}-{0}.csv'.format(str(date)[0:4]),
            report_type='NOAA'
            )

    ds = xr.load_dataset(indir, decode_times=False)
    points = report()
    print('{}| {:02} | {:02} | {:02} |'.format(date, points['wind'].shape[0], points['hail'].shape[0], points['tornado'].shape[0] ))

  Date  |Wind|Hail|Torn|
--------|----|----|----|
20170501| 69 | 00 | 00 |
20170508| 01 | 06 | 01 |
20170517| 109 | 20 | 07 |
20170518| 139 | 54 | 10 |
20170519| 19 | 22 | 14 |
20170524| 10 | 00 | 11 |
20170525| 17 | 04 | 00 |
20170530| 05 | 12 | 00 |
20180502| 77 | 37 | 20 |
20180504| 113 | 01 | 03 |
20180509| 29 | 08 | 00 |
20180512| 11 | 13 | 00 |
20180519| 04 | 16 | 01 |
20180524| 24 | 16 | 00 |
20180530| 24 | 19 | 03 |
20190513| 02 | 06 | 00 |
20190518| 32 | 10 | 00 |
20190521| 10 | 01 | 07 |
20190522| 18 | 33 | 21 |
20200520| 41 | 18 | 06 |
20200521| 05 | 51 | 01 |
20200526| 08 | 11 | 02 |
20200528| 01 | 01 | 03 |
20210506| 14 | 00 | 00 |
20210510| 03 | 26 | 00 |
20210513| 01 | 13 | 01 |
20210514| 14 | 39 | 01 |
20210521| 04 | 01 | 00 |
20210523| 21 | 10 | 00 |
20210524| 01 | 14 | 01 |
20210525| 18 | 20 | 00 |


# 2-6 Hr Test Dates

In [6]:
FRAMEWORK='POTVIN'; TIMESCALE='2to6'
base_path = f'/work/samuel.varga/data/{TIMESCALE}_hr_severe_wx/{FRAMEWORK}'
X,y,metadata = load_ml_data(base_path=base_path, 
                            mode='test', 
                            target_col='hail_severe__36km',
                           FRAMEWORK=FRAMEWORK,
                           TIMESCALE=TIMESCALE)

In [7]:
dates=np.sort(pd.unique(metadata['Run Date'])) #Grab the dates in the testing set
init_time_='0000' #20200519, 20180503, 20180511,  20180523, 20190520, 20210505, 20210520, 
dates

array(['20180503', '20180509', '20180511', '20180523', '20190501',
       '20190508', '20190518', '20190520', '20190523', '20190525',
       '20200504', '20200506', '20200513', '20200519', '20200520',
       '20200521', '20200522', '20200528', '20210505', '20210512',
       '20210518', '20210520', '20210521', '20210524', '20210526'],
      dtype=object)

In [ ]:
'20170501', '20170508', '20170517', '20170518', '20170519',
       '20170524', '20170525', '20170530', '20180502', '20180504',
       '20180509', '20180512', '20180519', '20180524', '20180530',
       '20190513', '20190518', '20190521', '20190522', '20200520',
       '20200521', '20200526', '20200528', '20210506', '20210510',
       '20210513', '20210514', '20210521', '20210523', '20210524',
       '20210525']

In [ ]:
print('{:^8}|{}|{}|{}|'.format('Date','Wind','Hail','Torn'))
print('--------|----|----|----|')
for date in dates:
    indir = glob(f'/work/mflora/SummaryFiles/{date}/{init_time_}/wofs_ENS_24*')[0]
    # Get the storm reports. 
    comps = decompose_file_path(indir)
    init_time = comps['VALID_DATE']+comps['VALID_TIME']

    init_time = comps['VALID_DATE']+comps['INIT_TIME']



    report = StormReportLoader(initial_time=init_time, 
            forecast_length=240,
            err_window=15, 
            reports_path='/work/samuel.varga/data/2to6_hr_severe_wx/reports/STORM_EVENTS_{0}-{0}.csv'.format(str(date)[0:4]),
            report_type='NOAA'
            )

    ds = xr.load_dataset(indir, decode_times=False)
    points = report()
    print('{}| {:02} | {:02} | {:02} |'.format(date, points['wind'].shape[0], points['hail'].shape[0], points['tornado'].shape[0] ))

  Date  |Wind|Hail|Torn|
--------|----|----|----|
20180503| 04 | 04 | 05 |
20180509| 33 | 09 | 00 |
20180511| 00 | 04 | 00 |
20180523| 11 | 31 | 01 |
